## EIA International

In [15]:
import pandas as pd
import urllib.request
import json
import zipfile
import time
import os

In [13]:
# download du fichier international .zip sur eia (https://www.eia.gov/opendata/)

# url of the international file on eia
url = "https://www.eia.gov/opendata/bulk/INTL.zip"


#/Users/alexandrebernard/Documents/Perso/Data/D4G/shiftdataportal_data
#defining file name and destination_raw
destination_raw = "../../data/_raw/eia/"
file_name = "eia_international_bulk.zip"

### Bulk Download of the txt file containing all data of International API route

In [2]:



#download the file from url
urllib.request.urlretrieve(url, destination_raw + file_name)
print("download completed")


download completed


### unzip the file

In [3]:
# unzip the file and delete zip

#ouvrir le fichier en mode lecture
with zipfile.ZipFile(destination_raw + file_name, 'r') as zip_ref:
    #Extract file in same repertory
    zip_ref.extractall(destination_raw)
print("file unzipped")


file unzipped


### Extraction of the JSONs from txt file into a dataframe

In [4]:

start_time = time.time()

json_list = []

print("---open txt file---")
#open the file in reading mode
with open(destination_raw + "INTL.txt", 'r') as file_txt:

    count = 0                        # TO DELETE
    # iteration on each json in the txt file
    for row in file_txt:
        #delete blank and new line caracter
        row = row.strip()
        #load the txt of one row into a json. use of "json.loads()" as we are loading a string into a json file corresponding to the row
        data_json = json.loads(row)
        #insert in dataframe
        #df_temp = pd.DataFrame(data_json)
        #add to the list of jsons
        json_list.append(data_json)
        
        #management of execution time
        count += 1                   # TO DELETE
        if count % 5000 == 0:
            print("---iteration on %s row completed---" %count)
        #if stop == 20:              # TO DELETE
        #    break                   # TO DELETE

df = pd.DataFrame(json_list)

iteration_time = time.time()
print("--- iteration done in %s seconds ---" %(iteration_time - start_time))



---open txt file---
---iteration on 5000 row completed---
---iteration on 10000 row completed---
---iteration on 15000 row completed---
---iteration on 20000 row completed---
---iteration on 25000 row completed---
---iteration on 30000 row completed---
---iteration on 35000 row completed---
---iteration on 40000 row completed---
---iteration on 45000 row completed---
---iteration on 50000 row completed---
---iteration on 55000 row completed---
---iteration on 60000 row completed---
---iteration on 65000 row completed---
---iteration on 70000 row completed---
---iteration on 75000 row completed---
---iteration on 80000 row completed---
---iteration on 85000 row completed---
---iteration on 90000 row completed---
---iteration on 95000 row completed---
---iteration on 100000 row completed---
--- iteration done in 8.494067907333374 seconds ---


### formating of the df

In [5]:
#delete non valid data in "data" column
start_time = time.time()

#deletion of NaN values
df_structured = df.dropna(subset=["data"])

dropna_time = time.time()
print("--- dropna done in %s seconds ---" %(dropna_time - start_time))

#management of the columns "data" that is a list of lists
#extend "data" column by creating a nes row for each internal list
df_structured = df_structured.explode("data")

explode_time = time.time()
print("--- explode done in %s seconds ---" %(explode_time - dropna_time))

# transformation of the "data" columns (containing a list [year, value] into 2 columns for date and value
df_structured[["date", "value"]] = pd.DataFrame(df_structured["data"].tolist(), index=df_structured.index)
df_structured = df_structured.drop("data", axis=1)

tolist_time = time.time()
print("--- tolist done in %s seconds ---" %(tolist_time - explode_time))

#we only keep annual data
df_structured = df_structured[df_structured["f"]=="A"]

annual_time = time.time()
print("--- annual done in %s seconds ---" %(annual_time - tolist_time))

#split of concatenated columns
# for the "series_id" column
df_structured[["file_name", "product_id", "activity_id", "country_region_id", "unit_id", "frequency_id"]] = df_structured["series_id"].str.split('[.-]', expand=True)
# for the "name" column
#df_structured[["product_name", "country_region_name", "frequency_name"]] = df_structured["name"].str.split(',', expand=True)
#=> not working because there can be 2 until 7 ","

concatcol_time = time.time()
print("--- concatcol done in %s seconds ---" %(concatcol_time - annual_time))

#reset of indexes
df_structured = df_structured.reset_index(drop=True)

#df.to_csv(destination_raw + "test.csv")
df_structured.head()
# run time before keeping only annual data >8min20"

--- dropna done in 0.11472892761230469 seconds ---
--- explode done in 7.350605249404907 seconds ---
--- tolist done in 2.8920748233795166 seconds ---
--- annual done in 8.739773035049438 seconds ---
--- concatcol done in 17.1390540599823 seconds ---


,series_id,name,units,f,copyright,source,geography,start,end,last_updated,...,notes,childseries,date,value,file_name,product_id,activity_id,country_region_id,unit_id,frequency_id
0,INTL.4702-33-WP21-THP.A,"Population, IEO OECD - Europe, Annual",people in thousands,A,None,"EIA, U.S. Energy Information Administration",AUT+BEL+CHE+CSK+CZE+DDR+DEU+DEUW+DNK+ESP+EST+F...,1980,2022,2023-06-27T18:22:43-04:00,...,NaN,NaN,2022,594423.7219,INTL,4702,33,WP21,THP,A
1,INTL.4702-33-WP21-THP.A,"Population, IEO OECD - Europe, Annual",people in thousands,A,None,"EIA, U.S. Energy Information Administration",AUT+BEL+CHE+CSK+CZE+DDR+DEU+DEUW+DNK+ESP+EST+F...,1980,2022,2023-06-27T18:22:43-04:00,...,NaN,NaN,2021,590999.9092,INTL,4702,33,WP21,THP,A
2,INTL.4702-33-WP21-THP.A,"Population, IEO OECD - Europe, Annual",people in thousands,A,None,"EIA, U.S. Energy Information Administration",AUT+BEL+CHE+CSK+CZE+DDR+DEU+DEUW+DNK+ESP+EST+F...,1980,2022,2023-06-27T18:22:43-04:00,...,NaN,NaN,2020,590119.4048,INTL,4702,33,WP21,THP,A
3,INTL.4702-33-WP21-THP.A,"Population, IEO OECD - Europe, Annual",people in thousands,A,None,"EIA, U.S. Energy Information Administration",AUT+BEL+CHE+CSK+CZE+DDR+DEU+DEUW+DNK+ESP+EST+F...,1980,2022,2023-06-27T18:22:43-04:00,...,NaN,NaN,2019,588597.2637,INTL,4702,33,WP21,THP,A
4,INTL.4702-33-WP21-THP.A,"Population, IEO OECD - Europe, Annual",people in thousands,A,None,"EIA, U.S. Energy Information Administration",AUT+BEL+CHE+CSK+CZE+DDR+DEU+DEUW+DNK+ESP+EST+F...,1980,2022,2023-06-27T18:22:43-04:00,...,NaN,NaN,2018,586326.2566,INTL,4702,33,WP21,THP,A


### columns renaming and selection

In [14]:
#selection and reordering of the columns
df_eia = df_structured[["source", "file_name", "name", "country_region_id", "geography", "product_id", "activity_id", "date", "unit_id", "units", "value", "last_updated"]]

#colums renaming
df_eia.rename(columns={
    'units': 'unit_name',
    'name': 'product_region_freq_name',
    'date': 'year'
}, inplace=True)

/var/folders/pd/8mtfl9m11sbf2r8nhg8s0jc80000gp/T/ipykernel_6955/3888079977.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eia.rename(columns={


### store cleaned eia data in csv file + delete zip and unzipped file

In [16]:
df_eia.to_csv(destination_raw + "eia_cleaned.csv")

os.remove(destination_raw + "eia_international_bulk.zip")
os.remove(destination_raw + "INTL.txt")